<a href="https://colab.research.google.com/github/lauben2001/Levered_Equity_Bootstrap/blob/main/Moving_Block_Bootstrap_Lev_Index_Funds_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Moving Block Bootstrap - Leveraged S&P500 ETF**

In [ ]:
pip install arch

In [ ]:
import numpy as np
import pandas as pd
import random
from pandas_datareader import data as pdr
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import arch as arch
from arch.bootstrap import optimal_block_length

In [ ]:
#Importieren der Daten & Cleanen
path="/content/drive/MyDrive/Daten - Phyton /SPX Daily Returns.xlsx"
sheet_name= "Tabelle1"
df_SPX_returns =pd.read_excel(path,sheet_name)
df_SPX_returns = df_SPX_returns.drop(columns=['Date', 'Adj Close**'])
df_SPX_returns = df_SPX_returns.drop(0, axis=0)
df_returns = df_SPX_returns.rename(columns={'Return': 'Daily Returns SPX'})
df_returns

In [ ]:
print(df_SPX_returns.min())

Return    -0.119799
dtype: float64


In [ ]:
#Pick random x-Block
def BlockPick(df,block_size):
  max_number = len(df) - block_size
  rand_number_pick =random.randint(0, max_number)
  df_block = df.iloc[rand_number_pick: rand_number_pick+ block_size]
  return df_block

In [ ]:
#Kreeiert einen simulierten Pfad für die gegebene Zeitreihe
def SimulatePath(df,simulated_years, blocksize):
  number_of_picks = round((simulated_years*252) / blocksize)
  for i in range(number_of_picks):
    if i ==0:
      df_one_path = BlockPick(df, blocksize)
    else:
      df_one_path = pd.concat([df_one_path, BlockPick(df, blocksize)])
  return df_one_path

In [ ]:
def max_drawdown(returns: pd.DataFrame) -> float:
# Calculate the cumulative returns
  cum_returns = (1 + returns).cumprod()
# Calculate the running maximum
  running_max = np.maximum.accumulate(cum_returns)
  # Calculate the drawdown
  drawdown = (cum_returns / running_max) - 1
# Calculate the maximum drawdown
  max_drawdown = drawdown.min()
  return max_drawdown

# **Basis Szenario: Keine Restriktionen**

In [ ]:
blocksize= round(optimal_block_length(df_SPX_returns)["circular"].iloc[0])

#How many leverage factors do you want?
number_of_leverage_factors= 5
#1. Create  Containers for results
ret_1Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_2Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_3Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_5Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_10Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_15Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_20Y = [[]*0 for i in range(number_of_leverage_factors)]

#2. Create Paths
number_of_path = 500000
for j in range(number_of_path):
  df_path= SimulatePath(df= df_SPX_returns,simulated_years= 20, blocksize = blocksize)                 #Create one simumalted Path
  index = 0
  for lev in range(1,number_of_leverage_factors+1):                                                     #Für Leverage Faktor 1-5
    df_lev_path = df_path * lev                                                                       #Simulierte Tagesrenditen * Leverage Faktor
    df_cum_ret_path = (1 + df_lev_path).cumprod() - 1                                                     #Kumullierte Renditereihe

    print("Minimum des CumPaths " + str(lev)+": " +str(df_cum_ret_path.min()))
    #1. Year
    ret_1Y[lev-1].append(df_cum_ret_path.iloc[252, 0])
    #2. Year
    ret_2Y[lev-1].append(df_cum_ret_path.iloc[504, 0])
    #3. Year
    ret_2Y[lev-1].append(df_cum_ret_path.iloc[756, 0])
    #5. Year
    ret_5Y[lev-1].append(df_cum_ret_path.iloc[1260, 0])
    #10. Year
    ret_10Y[lev-1].append(df_cum_ret_path.iloc[2772, 0])
    #15. Year
    ret_15Y[lev-1].append(df_cum_ret_path.iloc[3780, 0])
    #20. Year
    ret_20Y[lev-1].append(df_cum_ret_path.iloc[-1, 0])


#Creation of all dataFrames
df_1Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_2Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_5Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_10Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_15Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_20Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])

list_lists = [ret_1Y,ret_2Y, ret_5Y, ret_10Y, ret_15Y, ret_20Y]
list_dfs = [df_1Y, df_2Y, df_5Y, df_10Y, df_15Y, df_20Y]

yrs = 0
for  ret in list_lists:
  ret_quantils = []
  index = 0
  for lev in range(1,number_of_leverage_factors+1):
    ret_quantils = []
    ret_quantil = [np.quantile(ret[index], 0.01),np.quantile(ret[index], 0.05),np.quantile(ret[index], 0.1),np.quantile(ret[index], 0.25), np.quantile(ret[index], 0.5),np.quantile(ret[index], 0.75),np.quantile(ret[index], 0.9),np.quantile(ret[index], 0.95),np.quantile(ret[index], 0.99)]
    col_name =  "Return LevFactor" + str(lev)
    list_dfs[yrs][col_name] = ret_quantil
    index = index +1
  yrs = yrs + 1


with pd.ExcelWriter(r'/content/drive/MyDrive/Daten2_SimulationResults.xlsx') as writer:
  df_1Y.to_excel(writer, sheet_name='Returns 1Y')
  df_2Y.to_excel(writer, sheet_name='Returns 2Y')
  df_5Y.to_excel(writer, sheet_name='Returns 5Y')
  df_10Y.to_excel(writer, sheet_name='Returns 10Y')
  df_15Y.to_excel(writer, sheet_name='Returns 15Y')
  df_20Y.to_excel(writer, sheet_name='Returns 20Y')

#yrs = 0
#for  ret in list_lists:
#  ret_quantils = []
#  index = 0
#  for lev in range(1,number_of_leverage_factors+1):
#    if lev ==1 :
#      ret_quantils = []
#      ret_quantils = [np.quantile(ret[index], 0.01),np.quantile(ret[index], 0.05),np.quantile(ret[index], 0.1),np.quantile(ret[index], 0.25), np.quantile(ret[index], 0.5),np.quantile(ret[index], 0.75),np.quantile(ret[index], 0.9),np.quantile(ret[index], 0.95),np.quantile(ret[index], 0.99)]
#      col_name =  "Return LevFactor" + str(lev)
#      list_dfs[yrs][col_name] = ret_quantils
#    else:
#      ret_2quantils = []
#      ret_2quantils = [ret[lev-1,ret[0].index(ret_quantils[0]])]

#      col_name =  "Return LevFactor" + str(lev)
#      list_dfs[yrs][col_name] = ret_2quantils
#    index = index +1
#  yrs = yrs + 1



#2ter DataFrames -->Pfade von Lev 1 Quantilen










#**NegativSzenario: mind. -10% im Ersten Jahr**

In [ ]:
blocksize= round(optimal_block_length(df_SPX_returns)["circular"].iloc[0])

#How many leverage factors do you want?
number_of_leverage_factors= 5
#1. Create  Containers for results
ret_1Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_2Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_3Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_5Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_10Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_15Y = [[]*0 for i in range(number_of_leverage_factors)]
ret_20Y = [[]*0 for i in range(number_of_leverage_factors)]

#2. Create Paths
number_of_path = 10000
passed_paths = 0
while passed_paths < number_of_path:
  df_path= SimulatePath(df= df_SPX_returns,simulated_years= 20, blocksize = blocksize)                 #Create one simumalted Path
  df_cum_ret_path = (1 + df_path).cumprod() - 1
  if df_cum_ret_path.iloc[252, 0] < -0.1:
    passed_paths = passed_paths +1
    index = 0
    print(passed_paths)
    for lev in range(1,number_of_leverage_factors+1):                                                     #Für Leverage Faktor 1-5
      df_lev_path = df_path * lev                                                                       #Simulierte Tagesrenditen * Leverage Faktor
      df_cum_ret_path = (1 + df_lev_path).cumprod() - 1                                                     #Kumullierte Renditereihe

      #print("Minimum des CumPaths " + str(lev)+": " +str(df_cum_ret_path.min()))
      #1. Year
      ret_1Y[lev-1].append(df_cum_ret_path.iloc[252, 0])
      #2. Year
      ret_2Y[lev-1].append(df_cum_ret_path.iloc[504, 0])
      #3. Year
      ret_2Y[lev-1].append(df_cum_ret_path.iloc[756, 0])
      #5. Year
      ret_5Y[lev-1].append(df_cum_ret_path.iloc[1260, 0])
      #10. Year
      ret_10Y[lev-1].append(df_cum_ret_path.iloc[2772, 0])
      #15. Year
      ret_15Y[lev-1].append(df_cum_ret_path.iloc[3780, 0])
      #20. Year
      ret_20Y[lev-1].append(df_cum_ret_path.iloc[-1, 0])

#Creation of all dataFrames
df_1Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_2Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_5Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_10Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_15Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])
df_20Y = pd.DataFrame(index = ["99%", "95%","90%", "75%", "50%","25%","10%", "5%", "1%"])

list_lists = [ret_1Y,ret_2Y, ret_5Y, ret_10Y, ret_15Y, ret_20Y]
list_dfs = [df_1Y, df_2Y, df_5Y, df_10Y, df_15Y, df_20Y]

yrs = 0
for  ret in list_lists:
  ret_quantils = []
  index = 0
  for lev in range(1,number_of_leverage_factors+1):
    ret_quantils = []
    ret_quantil = [np.quantile(ret[index], 0.01),np.quantile(ret[index], 0.05),np.quantile(ret[index], 0.1),np.quantile(ret[index], 0.25), np.quantile(ret[index], 0.5),np.quantile(ret[index], 0.75),np.quantile(ret[index], 0.9),np.quantile(ret[index], 0.95),np.quantile(ret[index], 0.99)]
    col_name =  "Return LevFactor" + str(lev)
    list_dfs[yrs][col_name] = ret_quantil
    index = index +1
  yrs = yrs + 1


with pd.ExcelWriter(r'/content/drive/MyDrive/NegativSzenario(-10%)_Simulation.xlsx') as writer:
  df_1Y.to_excel(writer, sheet_name='Returns 1Y')
  df_2Y.to_excel(writer, sheet_name='Returns 2Y')
  df_5Y.to_excel(writer, sheet_name='Returns 5Y')
  df_10Y.to_excel(writer, sheet_name='Returns 10Y')
  df_15Y.to_excel(writer, sheet_name='Returns 15Y')
  df_20Y.to_excel(writer, sheet_name='Returns 20Y')

print("DONE!")